In [1]:
import os
from nbed.driver import NbedDriver
import numpy as np

from nbed.ham_builder import HamiltonianBuilder
from openfermion import get_sparse_operator
import scipy as sp
    
from pyscf import gto, scf, fci, cc

/home/lex/anaconda3/envs/embed/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
# get xyz file for water
notebook_dir = os.getcwd()
projects_dir = os.path.dirname(notebook_dir)
NBed_dir = os.path.dirname(projects_dir)
Test_dir = os.path.join(NBed_dir, 'tests')
mol_dir = os.path.join(Test_dir, 'molecules')

water_xyz_path = os.path.join(mol_dir, 'water.xyz')

In [3]:
geometry_path = os.path.join(mol_dir, 'water.xyz')
# geometry_path = os.path.join(mol_dir, 'H4.xyz')

In [4]:
geometry_path

'/home/lex/Documents/PhD/Nbed/tests/molecules/water.xyz'

In [5]:
with open(geometry_path, 'r') as infile:
    xyz_string = infile.read()
    
print(xyz_string)

3

O   0.00000  0.0000000  0.1653507
H   0.00000  0.7493682 -0.4424329
H   0.00000 -0.7493682 -0.4424329 


In [6]:
# options
geometry = xyz_string
n_active_atoms=1
basis = 'STO-3G'
xc_functional = 'lda, vwn'# 'lda, vwn' #'B3LYP'
run_virtual_localization = False

run_fci_emb = True
run_ccsd_emb = False
max_ram_memory = 8_000
_init_huzinaga_rhf_with_mu = False

max_hf_cycles=5000


projector = 'both'
localization = 'ibo' # spade, ibo
occupied_threshold = 0.95
virtual_threshold = 0.95

In [7]:
driver = NbedDriver(geometry = geometry,
                    n_active_atoms=n_active_atoms,
                    basis = basis,
                    xc_functional = xc_functional,
                    run_virtual_localization = run_virtual_localization,
                    run_fci_emb = run_fci_emb,
                    run_ccsd_emb = run_ccsd_emb,
                    max_ram_memory = max_ram_memory,
                    _init_huzinaga_rhf_with_mu = _init_huzinaga_rhf_with_mu,
                    max_hf_cycles=max_hf_cycles,
                    occupied_threshold=occupied_threshold,
                    projector = projector,
                    localization = localization)

 Iterative localization: IB/P4/2x2, 6 iter; Final gradient 8.60e-12


In [8]:
global_DFT_energy = driver._global_rks.e_tot
global_FCI_energy = driver._global_fci.e_tot

print(f'global FCI energy: {global_FCI_energy}')
print(f'global DFT energy: {global_DFT_energy}')

global FCI energy: -75.01535247594691
global DFT energy: -74.73474041595246


In [9]:
qubit_transform = 'jordan_wigner'

    
huz_classical = driver._huzinaga['classical_energy']
rhf_huz = driver._huzinaga["scf"]
qham_huz = HamiltonianBuilder(
                        scf_method=rhf_huz,
                        constant_e_shift=huz_classical,
                        transform=qubit_transform,
                        ).build(n_qubits=None, taper=False)

mu_classical = driver._mu['classical_energy']
rhf_mu = driver._mu["scf"]
qham_mu = HamiltonianBuilder(
                        scf_method=rhf_mu,
                        constant_e_shift=mu_classical,
                        transform=qubit_transform,
                        ).build(n_qubits=None, taper=False)

In [10]:
huz_H_sparse = get_sparse_operator(qham_huz)
huz_e_emb = sp.sparse.linalg.eigsh(huz_H_sparse, k=1, which='SA')[0][0]
huz_e_emb

mu_H_sparse = get_sparse_operator(qham_mu)
mu_e_emb = sp.sparse.linalg.eigsh(mu_H_sparse, k=1, which='SA')[0][0]
mu_e_emb

-75.11791105247659

In [11]:
huz_e_emb - driver._huzinaga['e_fci']

-8.668621376273222e-13

In [12]:
mu_e_emb - driver._mu['e_fci']

7.105427357601002e-13

In [13]:
print(driver.localized_system.active_MO_inds)
print(driver.localized_system.enviro_MO_inds)

[0 3 4]
[1 2]


In [14]:
print(f'global DFT error: {abs(global_FCI_energy - global_DFT_energy)}')

print(f'mu error: {abs(global_FCI_energy - huz_e_emb)}')
print(f'huz error: {abs(global_FCI_energy - mu_e_emb)}')

global DFT error: 0.2806120599944535
mu error: 0.1025567269777099
huz error: 0.10255857652967393


In [15]:
# how subsystem was selected (either % active AOs [pyscf] or singular values[SPADE]!)
driver.localized_system.enviro_selection_condition

array([0.95812183, 0.60058968, 0.60058968, 0.99995469, 0.99276452])

In [16]:
len(list(qham_huz))

276

In [17]:
len(list(qham_mu))

276

In [18]:
driver._global_rks.e_tot

-74.73474041595246

In [19]:
driver.embed_dft_in_dft('B3LYP') # wb97m_v

In [20]:
print(driver._mu_dft['e_rks_cheap'])
print(driver._mu_dft['e_rks_expen'])

-74.73474211513509
-75.28370184457098


In [21]:
print(driver._huzinaga_dft['e_rks_cheap'])
print(driver._huzinaga_dft['e_rks_expen'])

-74.7347404158168
-75.28370013133782


In [22]:
driver._global_rks.e_tot - driver._mu_dft['e_rks_cheap']

1.6991826328194293e-06

In [23]:
driver._global_rks.e_tot - driver._huzinaga_dft['e_rks_cheap']

-1.3565681911131833e-10

In [24]:
mu_dft_in_dft_error =  abs(global_FCI_energy -  driver._mu_dft['e_rks_expen'])
print(f'mu error: {mu_dft_in_dft_error}')

huz_dft_in_dft_error =  abs(global_FCI_energy -  driver._huzinaga_dft['e_rks_expen'])
print(f'huz error: {huz_dft_in_dft_error}')

mu error: 0.2683493686240723
huz error: 0.26834765539091165


In [25]:
cccc = driver._huzinaga_dft['scf_cheap_dft'].mo_coeff
FF = driver._huzinaga_dft['scf_cheap_dft'].get_fock()
np.around(cccc.T @ FF @ cccc, 2)

array([[-18.18,   0.  ,   0.  ,   0.  ,  -0.  ,  -0.  ,   0.  ],
       [  0.  ,  -0.29,  -0.  ,  -0.  ,   0.  ,   0.  ,  -0.  ],
       [  0.  ,  -0.  ,  -0.06,   0.  ,   0.  ,  -0.  ,   0.  ],
       [  0.  ,  -0.  ,   0.  ,   0.31,  -0.  ,  -0.  ,   0.  ],
       [ -0.  ,   0.  ,   0.  ,  -0.  ,   0.38,   0.  ,   0.  ],
       [ -0.  ,   0.  ,  -0.  ,  -0.  ,   0.  ,   0.41,   0.  ],
       [  0.  ,  -0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.79]])

In [26]:
driver._huzinaga_dft['scf_cheap_dft'].converged

True

In [1]:
## can see which orbitals are removed (once env is removed)
# and their associated energies:

In [28]:
print(driver._mu['mo_energies_emb_pre_del'])
print(driver._mu['mo_energies_emb_post_del'])

[-2.01401285e+01 -6.71236840e-01 -3.60832604e-01  5.83897484e-01
  7.22265114e-01  1.99999950e+06  1.99999992e+06]
[-20.1401285   -0.67123684  -0.3608326    0.58389748   0.72226511]


In [29]:
print(driver._huzinaga['mo_energies_emb_pre_del'])
print(driver._huzinaga['mo_energies_emb_post_del'])

[-20.14012841  -0.6712376   -0.36083328   0.08139568   0.49870932
   0.58389757   0.72226471]
[-20.14012841  -0.6712376   -0.36083328   0.58389757   0.72226471]
